In [1]:
import re

import numpy as np
import tensorstore as ts

In [2]:
bucket = 'mpi-j0251-ae38dfb91896253ecdb1033161e29f75'
image = 'j0251_72/raw_clahe/'
segments = 'j0251_72_mito_sj_vc_gt_20220811/'
size = (300, 300, 150)

In [3]:
patt = re.compile("bounding_box { start { x: (\d+) y: (\d+) z: (\d+) } size { x: (\d+) y: (\d+) z: (\d+) } }")
boxes = []
with open("bounding_box.txt") as f:
    for line in f:
        match = re.match(patt, line)
        coords = tuple(int(c) for c in match.groups())
        assert coords[3:] == size
        boxes.append(coords[:3])

In [4]:
def open_ds(path):
    dataset_future = ts.open({
        'driver': 'neuroglancer_precomputed',
        'kvstore': f'gs://{bucket}/{path}',
        'context': { 'cache_pool': { 'total_bytes_limit': 100_000_000 } },                                                                                            
        'recheck_cached_data': 'open',
    })
    return dataset_future.result()

image_ds = open_ds(image)
segments_ds = open_ds(segments)

In [13]:
for box_idx, box in enumerate(boxes):
    print(box_idx)
    slices = tuple(slice(b, b+e) for b,e in zip(box, size))
    img = image_ds[slices].read().result()
    seg = segments_ds[slices].read().result()
    np.savez(f"data/box_{box_idx:02}", img=img, seg=seg)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
